##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys
def find_repo_root():
    try:
        script_path = os.path.abspath('.')
    except NameError:
        script_path = os.path.abspath(__file__)
    result = None
    while True:
        script_path = os.path.dirname(script_path)
        if os.path.isdir(os.path.join(script_path, '.git')):
            result = script_path
        if script_path == '/':
            return result
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)

## <i class="fa fa-wrench"></i> 安装必要依赖和工具

In [ ]:
import subprocess

libraries_to_install = [
    
]

tools_to_install = [
    'wget',
    'iputils-ping'
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")

# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-folder-plus"></i> 创建Content文件夹
创建Content文件夹来还原Colab的运行环境，以减少代码总体修改量

In [ ]:
import os

def check_environment(output):
    environments = {
        "AutoDL": {
            "content_path": "/root/autodl-tmp/content",
            "env_name": "AutoDL"
        },
        "OpenBayes": {
            "content_path": "/openbayes/home/content",
            "env_name": "OpenBayes"
        }
    }
    # 检查输出是否包含字符串"autodl"或"openbayes"
    # for i in output:
    if "AutoDL" in output:
        return environments["AutoDL"]
    elif "OpenBayes" in output:
        return environments["OpenBayes"]
    # 如果输出中没有包含上述字符串，则返回空字典
    return {}


def detect_environment():
    # 初始化 content_path 和 env_name 变量
    content_path = None
    env_name = None

    # 将命令行存储在列表中
    commands = [
        "cd /openbayes/home && chmod +x /etc/welcome && /etc/welcome",
        "chmod +x /etc/autodl-motd && /etc/autodl-motd"
    ]

    # 遍历命令行列表，执行命令并存储输出
    for command in commands:
        try:
            import subprocess
            # 使用 subprocess 模块执行命令
            r = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
            output = r.stdout
            # 将输出转换为字符串
            output = output.decode()
            # print(output)
            result = check_environment(output)
            content_path = result["content_path"]
            env_name = result["env_name"]
            # 如果检测到环境，则退出循环
            if content_path and env_name:
                break
        except Exception as e:
            # print("无法执行命令：", e)
            continue

    # 打印结果
    if content_path and env_name:
        print("当前运行环境：", env_name)
        print("内容路径：", content_path)
    else:
        print("未检测到当前运行环境")
        
    return {
        "content_path":content_path,
        "env_name":env_name
    }


content_path=detect_environment()["content_path"]
# 在根目录下创建Content文件夹
content_folder = content_path
if not os.path.exists(content_folder):
    os.makedirs(content_folder)

## <i class="fa fa-download"></i> 下载原笔记
Temp文件夹并下载对应笔记到Temp目录下

In [ ]:
file_name = "Live3D-v2.ipynb"
url = "https://github.com/transpchan/Live3D-v2/raw/main/notebook.ipynb"

import os, sys

from func.env import setProxyCLI, proxyWget

cb = setProxyCLI()
proxy = cb["proxy"]
region = cb["region"]
proxyURL = cb["proxyURL"]

# 在当前运行的ipynb文件所在的目录下创建临时文件夹temp
def create_temp_folder(temp_folder):
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)

# 获取当前运行的 ipynb 文件所在的目录
cwd = os.getcwd()

# 在该目录下创建临时文件夹 temp
temp_folder = os.path.join(cwd, "temp")

# 如果temp文件夹不存在，则创建文件夹
create_temp_folder(temp_folder)

# 覆盖下载指定的文件到 temp 文件夹中

fastDreamBoothPath = os.path.join(temp_folder, file_name)
if os.path.exists(fastDreamBoothPath):
    !rm -rf $fastDreamBoothPath

out = os.path.join(temp_folder, file_name)

# Execute the command with a timeout of 5 seconds
!$proxy && wget -O $out $url

# Check the exit status
status = !echo $?
if int(status[0]) != 0:
    print("下载失败")
else:
    print("下载成功")

## <i class="fa fa-broom"></i> 自动适配笔记
清理Notebook中的输出，对Colab版中的路径进行替换，以适配openbayes或autodl的路径

In [ ]:
import json
import os
import re


def modify_file_name(file_path):
    # 获取文件名和文件扩展名
    file_name, file_ext = os.path.splitext(file_path)
    # 在文件名后加上 -modified
    modified_file_path = file_name + "-modified" + file_ext
    return modified_file_path


def replace_root_path(notebook: str, old_root: str, new_root: str) -> str:
    # 加载notebook
    with open(notebook, "r") as f:
        nb = json.load(f)

    env_name=detect_environment()["env_name"]
    
    # 指定需要删除的单元格，根据一定的特征来进行识别删除
    if env_name=="OpenBayes":
        delList = [
            # '@markdown #Instance Images',
            # '@markdown #Concept Images',
        ]
    elif env_name=="AutoDL":
        delList = [

        ]        
    
    # 由于 Python 中的 for 循环是基于迭代器的，所以如果在循环过程中对列表进行修改（如在这里的删除操作），可能会导致结果不如预期。
    for cell in nb['cells'][:]: # 在 Python 中，[:] 可以用来复制一个列表
        delete_cell = False
        for line in cell['source']:
            if any(keyword in line for keyword in delList):
                delete_cell = True
                break
        if delete_cell:
            nb['cells'].remove(cell)

    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            # Initialize the flag variable
            import_added = False
            # 隐藏代码
            cell["metadata"] = {"collapsed": True}
            # 清除代码单元格的执行次数
            cell["execution_count"] = None
            # 清除代码单元格的输出
            cell["outputs"] = []
            
            for i, line in enumerate(cell["source"]):
                # 添加一个条件判断，只有当该行不是网址路径时才进行替换
                if not line.startswith(("http", "https")):
                    # 替换旧的根路径为新的根路径
                    cell["source"][i] = line.replace(old_root, new_root)
                # 把引入google.colab的库的代码行标注为注释
                if "google.colab" in line:
                    cell["source"][i] = "# " + line
                # 把使用google drive盘挂载的代码行标注为注释
                if "drive.mount" in line:
                    cell["source"][i] = "# " + line
                # 如果找到 "!wget" 字符串，就将其替换为 "!proxy && wget"
                if "!wget" in line:
                    cell["source"][i] = line.replace("!wget", "!$proxy && wget")
                # 定义 proxyWget 函数（请确保已经定义了 proxyWget 函数）
                
                import textwrap

                if "wget.download" in line:
                    # 使用正则表达式查找所有使用了 proxyWget 函数的行
                    match = re.search(r"wget\.download\(.+\)", line)
                    if match:
                        # 获取 wget.download 函数调用的参数
                        params = match.group().strip("wget.download()")
                        # 计算新的一行代码的缩进
                        indent = " " * line.count(" ")
                        # 将 wget.download 函数调用替换为带有新参数的函数调用
                        new_line = (
                            f"{indent}proxyWget({params}, os.getcwd(), '{proxyURL}')\n"
                        )
                        cell["source"][i] = new_line

            for i, line in enumerate(cell["source"]):
                # 替换@param{type: 'xxx'}中的单引号为双引号
                if "@param" in line:
                    cell["source"][i] = line.replace("'", '"')

            for i, line in enumerate(cell["source"]):
                # Check if the line contains "wget" and add the import statement if necessary
                import textwrap
                if "wget" in line and not import_added:
                    import_statements = [
'''import os,sys
def find_repo_root():
    try:
        script_path = os.path.abspath('.')
    except NameError:
        script_path = os.path.abspath(__file__)
    result = None
    while True:
        script_path = os.path.dirname(script_path)
        if os.path.isdir(os.path.join(script_path, '.git')):
            result = script_path
        if script_path == '/':
            return result
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
'''
                    ]
                    import_statements_with_newline = [
                        s + "\n" for s in import_statements
                    ]
                    
                    # print(import_statements_with_newline)
                    cell["source"] = import_statements_with_newline + cell["source"]
                    # Set the flag to indicate that the import has been added
                    import_added = True
                    
            for i, line in enumerate(cell["source"]):    
                # 把pip安装的静默去掉
                if "pip install -q" in line:
                    cell["source"][i] = line.replace("pip install -q", "pip install")
                # 把wget的静默去掉
                if "wget -q" in line:
                    cell["source"][i] = line.replace("wget -q", "wget")

                # # 原始字符串
                # original = "!cp -r /openbayes/home/content/usr/local/lib/python3.8/dist-packages /usr/local/lib/python3.8/"
                # if original in line:
                #     # 替换python路径
                #     cell["source"][i] = re.sub(r'/usr/local/lib/python3.8/', '/usr/local/lib/python/', original)
                
    # 遍历所有单元格，去掉科学计数法表示的数字的双引号，并且保留每个数字两边的中括号
    
    # 定义正则表达式
    pattern = r'"([\d.]+e[+-]\d+)"'  
    for cell in nb['cells']:
        # 如果单元格是代码单元格
        if cell['cell_type'] == 'code':
            # 遍历所有行
            for i, line in enumerate(cell['source']):
                # 在整个字符串中匹配正则表达式
                match = re.search(pattern, line)
                # 如果匹配成功
                if match:
                    # 将文本中的双引号去掉
                    new_line = re.sub(pattern, r'\1', line)
                    
                    # print(line)  # 输出原来的行
                    # print(new_line)  # 输出修改后的行
                    
                    # 将文本替换回原来的位置
                    cell['source'][i] = new_line

    # 保存修改后的notebook
    newNotebook = modify_file_name(notebook)
    with open(newNotebook, "w") as f:
        json.dump(nb, f)

    return newNotebook

newNotebookPath = replace_root_path(
    notebook=fastDreamBoothPath, old_root="/content", new_root=content_folder
)

# #对newNotebookPath进行格式化处理
# !black $newNotebookPath

print(newNotebookPath)

## <i class="fa fa-plus-circle"></i> 批量插入前置Cell
批量找到特征字符串的Cell之后，在这个Cell前面添加多一个Cell，而前一个Cell的内容则由另一个参数传入

In [ ]:
import json
import copy

def add_cell_before_with_specific_string(path_to_modified, modify_points):
    """
    批量找到特征字符串的Cell之后，在这个Cell前面添加多一个Cell，而前一个Cell的内容则由另一个参数传入
    
    Args:
        path_to_modified: (str) 要修改的文件的路径
        modify_points: (list[dict]) 每个要修改的点的信息,每个点需要传入三个参数,
                      new_cell_content: (str)新增加的cell的内容
                      feature_string: (str)特征字符串
                      cell_type: (str) cell类型,可能是'code','markdown','text'
    """
    with open(path_to_modified, 'r') as f:
        modified = json.load(f)

    for point in modify_points:
        new_cell_content = point['new_cell_content']
        feature_string = point['feature_string']
        cell_type = point['cell_type']
        for index, cell_modified in enumerate(modified['cells']):
            for line_modified in cell_modified['source']:
                if feature_string in line_modified:
                    modified['cells'].insert(index, {"cell_type": cell_type, "metadata": {}, "source": [new_cell_content]})
                    print(f'成功在包含特征字符串 "{feature_string}" 的代码单元格之前添加新的单元格')
                    break
            if feature_string in line_modified:
                break
    with open(path_to_modified, 'w') as f:
        json.dump(modified, f)

# 示例：修改被修改对象文件，使用代码剪贴板文件中的内容
modify_points = [
    {
        'new_cell_content': '%cd /openbayes/home/content/',
        'feature_string': '#@title Download the code',
        'cell_type': 'code'
    },
]

add_cell_before_with_specific_string(
    path_to_modified=newNotebookPath,
    modify_points=modify_points
)